In [ ]:
import xarray as xr
import xpublish
import zarr

import numpy as np
from json import loads

from fsspec.implementations.http import HTTPFileSystem

from xarray.testing import assert_identical, assert_equal, assert_chunks_equal

In [ ]:
fs = HTTPFileSystem()
http_map = fs.get_mapper('http://0.0.0.0:9000')

In [ ]:
for key in ['.zmetadata', 'keys']:
    print(key, http_map[key], '\n')

In [ ]:
d = loads(http_map['.zmetadata'])
d['metadata']['air/.zattrs']

In [ ]:
zg = zarr.open_consolidated(http_map, mode='r')

In [ ]:
zg.tree()

In [ ]:
ds = xr.open_zarr(http_map, consolidated=True, decode_cf=True)
ds

In [ ]:
ds_tutorial = xr.tutorial.open_dataset(
    'air_temperature',
    chunks=dict(lat=5, lon=5),
    decode_cf=True)

In [ ]:
ds_tutorial.air.attrs

In [ ]:
def test(actual, expected, index):
    '''a simple equality test with index as a parameter'''
    assert np.array_equal(actual[index].values, expected[index].values)

In [ ]:
# test a bunch of indexing patterns
for index in [(0, 0, 0),
              (slice(0, 4), 0, 0),
              (slice(0, 4), slice(0, 4), 0),
              (slice(0, 4), slice(0, 4), slice(0, 4)),
              (slice(-4), slice(0, 4), slice(0, 4)),
              (slice(None), slice(0, 4), slice(0, 4)),
              (slice(None), slice(None), slice(0, 4)),
              (slice(None), slice(None), slice(None)),
    ]:
    print(index)
    test(ds_tutorial['air'], ds['air'], index)

In [ ]:
assert_equal(ds, ds_tutorial)
assert_chunks_equal(ds, ds_tutorial)
assert_identical(ds, ds_tutorial)